In [3]:
import scipy.sparse as sps
from Recommenders.Hybrid.ItemUserHybridKNNRecommender import ItemUserHybridKNNRecommender
from Utils.Evaluator import EvaluatorHoldout
import Utils.Reader as Reader
from Data_manager.split_functions.split_train_validation_random_holdout import\
    split_train_in_two_percentage_global_sample
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Utils.Evaluator import EvaluatorHoldout
import Utils.Reader as Reader

from Data_manager.split_functions.split_train_validation_random_holdout import\
    split_train_in_two_percentage_global_sample
#URM_all=Reader.load_URM(values_to_replace={0.01:0.02})
URM_all=Reader.read_train_csr(values_to_replace={0:0.03})
group_id=20
alpha=0.4
beta=0.4
gamma=0.8
b_factor=0.04
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GeneralizedMergedHybridRecommender import GeneralizedMergedHybridRecommender
import numpy as np
profile_length = np.ediff1d(sps.csr_matrix(URM_all).indptr)
sorted_users = np.argsort(profile_length)
block_size = int(len(profile_length)*b_factor)
start_pos = group_id*block_size
end_pos = min((group_id+1)*block_size, len(profile_length))

users_in_group = sorted_users[start_pos:end_pos]

users_in_group_p_len = profile_length[users_in_group]
users_not_in_group_flag = np.isin(sorted_users, users_in_group, invert=True)
users_not_in_group = sorted_users[users_not_in_group_flag]


rp3=RP3betaRecommender(URM_all)
rp3.fit()

ials=IALSRecommender(URM_all)
ials.fit()

itemKNN=ItemKNNCFRecommender(URM_all)
itemKNN.fit()

higher_recommender = GeneralizedMergedHybridRecommender(
    URM_train=URM_all,
    recommenders=[
        rp3,
        ials,
        itemKNN

    ],
    verbose=False
)
higher_recommender.fit(
    alphas=[
        alpha,beta,
        1 - alpha - beta
    ]
)

slim =SLIM_BPR_Cython(URM_all)
slim.fit()

p3=P3alphaRecommender(URM_all)
p3.fit()

lower_recommender = GeneralizedMergedHybridRecommender(
    URM_train=URM_all,
    recommenders=[
        p3,
        slim

    ],
    verbose=False
)
lower_recommender.fit(
    alphas=[
        gamma,
        1-gamma
    ]
)



Unique UserID in the URM are 41629
Unique UserID in the URM and ICM are 41629
5826506
RP3betaRecommender: Similarity column 24507 (100.0%), 1382.55 column/sec. Elapsed time 17.73 sec
IALSRecommender: Epoch 1 of 115. Elapsed time 15.23 sec
IALSRecommender: Epoch 2 of 115. Elapsed time 34.66 sec
IALSRecommender: Epoch 3 of 115. Elapsed time 50.28 sec
IALSRecommender: Epoch 4 of 115. Elapsed time 1.11 min
IALSRecommender: Epoch 5 of 115. Elapsed time 1.37 min
IALSRecommender: Epoch 6 of 115. Elapsed time 1.62 min
IALSRecommender: Epoch 7 of 115. Elapsed time 1.88 min
IALSRecommender: Epoch 8 of 115. Elapsed time 2.16 min
IALSRecommender: Epoch 9 of 115. Elapsed time 2.43 min
IALSRecommender: Epoch 10 of 115. Elapsed time 2.71 min
IALSRecommender: Epoch 11 of 115. Elapsed time 2.98 min
IALSRecommender: Epoch 12 of 115. Elapsed time 3.26 min
IALSRecommender: Epoch 13 of 115. Elapsed time 3.52 min
IALSRecommender: Epoch 14 of 115. Elapsed time 3.99 min
IALSRecommender: Epoch 15 of 115. Elaps

/bin/sh: free: command not found


Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Automatic selection of fastest train mode. Unable to get current RAM status, you may be using a non-Linux operating system. Using dense matrix.
Processed 41629 (100.0%) in 2.20 sec. BPR loss is 3.54E-03. Sample per second: 18912
SLIM_BPR_Recommender: Epoch 1 of 420. Elapsed time 1.26 sec
Processed 41629 (100.0%) in 0.62 sec. BPR loss is 1.02E-02. Sample per second: 67134
SLIM_BPR_Recommender: Epoch 2 of 420. Elapsed time 1.68 sec
Processed 41629 (100.0%) in 0.94 sec. BPR loss is 1.86E-02. Sample per second: 44497
SLIM_BPR_Recommender: Epoch 3 of 420. Elapsed time 1.99 sec
Processed 41629 (100.0%) in 1.23 sec. BPR loss is 2.56E-02. Sample per second: 33775
SLIM_BPR_Recommender: Epoch 4 of 420. Elapsed time 2.29 sec
Processed 41629 (100.0%) in 0.52 sec. BPR loss is 3.13E-02. Sample per second: 79499
SLIM_BPR_Recommender: Epoch 5 of 420. Elapsed time 2.58 sec
Processed 41629 (100.0%) in 0.82 sec. BPR loss is 4.15E

In [4]:
import os
import numpy as np
import csv
import pandas as pd
parent_dir =  "../"
read_path = os.path.join(parent_dir, "data/data_target_users_test.csv")

with open(read_path, "r") as f:
    dataFrame = pd.read_csv(f, skiprows=0)
    dataFrame.columns = ["UserID"]
    user_array = np.asarray(dataFrame["UserID"])

write_path = os.path.join(parent_dir, "Testing_Results/submission.csv")
print(user_array)
user_array = user_array.tolist()
with open(write_path, "w+") as d:

    writer = csv.writer(d)
    writer.writerow(['user_id', 'item_list'])


    for userID in user_array:
        if userID in users_in_group:
            writer.writerow([userID, str(np.array(higher_recommender.recommend(userID, 10)))[1:-1]])
        else:
            writer.writerow([userID, str(np.array(p3.recommend(userID, 10)))[1:-1]])

[    0     1     2 ... 41626 41627 41628]
